# Learning Goals
In Assignment 2, we learnt how to construct networks of spiking neurons and propagate information through a network of fixed weights. In this assignment, you will learn how to train network weights for a given task using brain-inspired learning rules.

Let's import all the libraries required for this assignment. 

In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt

# Question 1: Training a Network

## 1a. 
What is the purpose of a learning algorithm? In other words, what does a learning algorithm dictate, and what is the objective of it?

## Answer 1a. 
A learning algorithm (otherwise called a model) attempts to take some input and map it to some output. Now you might call this a function, and you would be correct, but a learning algorithm is merely a function approximator. Not all input/output combinations have underlying functions, some things in life are just random. Therefore, we try to build a function approximator that tries to minimize the error for this mapping. Thus given some input output pairs, we build a function approximator, that learns the function by minimizing the error between its predictions and the ground truth.

## 1b. 
Categorize and explain the various learning algorithms w.r.t. biological plausibility. Can you explain the tradeoffs involved with the different learning rules? *Hint: Think computational advantages and disadvantages of biological plausibility.*

## Answer 1b. 
Some of the more popular learning algorithm paradigms include
- Supervised Learning: Supervised Learning is the classic function approximator learning algorithm. We get inputs and outputs and we need to find a function that minimizes the error. The biological plausibility or supervised learning is not very strong since labeled data isn't something that's often seen in real life. However, one might make the argument that reflexes such as actions taken when you get a sensory input are a result of your brain using a model that is akin to supervised learning.
- Unsupervised Learning: For unsupervised learning, the model is not given any labels which you would imagine to be more similar to how real life works. Instead, the model exploits the underlying structure in the data to learn patterns. This is more biologically plausible (similar items are grouped together in the brain).
- Reinforcement Learning: Given a reward function and some start point, RL aims to maximize the reward in its environment by taking actions. This is also biologically plausible since the world and society in general has a set of unspoken reward functions which condition our behavior.
- Hebbian Learning: Hebbian learning is a type of unsupervised learning that is biologically plausible. It states that neurons that fire together wire together. This is a very simple learning rule that is often used in the brain to learn patterns. However, it is not very good at learning complex patterns.

# Question 2: Hebbian Learning

## 2a.

In this exercise, you will implement the hebbian learning rule to solve AND Gate. First, we need to create a helper function to generate the training data. The function should return lists of tuples where each tuple comprises of numpy arrays of rate-coded inputs and the corresponding rate-coded output. 

Below is the function to generate the training data. Fill the components to return the training data. 

In [2]:
def genANDTrainData(snn_timestep):
    """ 
    Function to generate the training data for AND 
        Args:
            snn_timestep (int): timesteps for SNN simulation
        Return:
            train_data (list): list of tuples where each tuple comprises of numpy arrays of rate-coded inputs and output
        
        Write the expressions for encoding 0 and 1. Then append all 4 cases of AND gate to the list train_data
    """
    
    #Initialize an empty list for train data
    train_data = []
    
    #encode 0. Numpy random choice function might be useful here. 
    z = np.random.choice([0,1], size=snn_timestep, p=[0.7, 0.3])
    
    #encode 1. Numpy random choice function might be useful here. 
    o = np.random.choice([0,1], size=snn_timestep, p=[0.3, 0.7])
    
    #Append all 4 cases of AND gate to train_data. Numpy stack operation might be useful here. 
    train_data.append((np.stack((z, z), axis=0), z))
    train_data.append((np.stack((z, o), axis=0), z))
    train_data.append((np.stack((o, z), axis=0), z))
    train_data.append((np.stack((o, o), axis=0), o))

    return train_data

## 2b. 
We will use the implementation of the network from assignment 2 to create an SNN comprising of one input layer and one output layer. Can you explain algorithmically, how you can use this simple architecture to learn AND gate. Your algorithm should comprise of encoding, forward propagation, network training, and decoding steps. 

## Answer 2b. 
- Encoding: We can encode the input using a rate encoding
- Architecture: We can use a simple 2 layer network with a single input and output layer
- Training: Training is simple, we just iterate over the samples until our predictions converge to a good point in the loss functions
- Decoding: We can decode the output using a simple thresholding function

The SNN has already been implemented for you. You do not need to do anything here. Just understand the implementation so that you can use it in the later parts. 

In [3]:
class LIFNeurons:
    """ 
        Define Leaky Integrate-and-Fire Neuron Layer 
        This class is complete. You do not need to do anything here.
    """

    def __init__(self, dimension, vdecay, vth):
        """
        Args:
            dimension (int): Number of LIF neurons in the layer
            vdecay (float): voltage decay of LIF neurons
            vth (float): voltage threshold of LIF neurons
        
        """
        self.dimension = dimension
        self.vdecay = vdecay
        self.vth = vth

        # Initialize LIF neuron states
        self.volt = np.zeros(self.dimension)
        self.spike = np.zeros(self.dimension)
    
    def __call__(self, psp_input):
        """
        Args:
            psp_input (ndarray): synaptic inputs 
        Return:
            self.spike: output spikes from the layer
                """
        self.volt = self.vdecay * self.volt * (1. - self.spike) + psp_input
        self.spike = (self.volt > self.vth).astype(float)
        return self.spike

class Connections:
    """ Define connections between spiking neuron layers """

    def __init__(self, weights, pre_dimension, post_dimension):
        """
        Args:
            weights (ndarray): connection weights
            pre_dimension (int): dimension for pre-synaptic neurons
            post_dimension (int): dimension for post-synaptic neurons
        """
        self.weights = weights
        self.pre_dimension = pre_dimension
        self.post_dimension = post_dimension
    
    def __call__(self, spike_input):
        """
        Args:
            spike_input (ndarray): spikes generated by the pre-synaptic neurons
        Return:
            psp: postsynaptic layer activations
        """
        psp = np.matmul(self.weights, spike_input)
        return psp
    
    
class SNN:
    """ Define a Spiking Neural Network with No Hidden Layer """

    def __init__(self, input_2_output_weight, 
                 input_dimension=2, output_dimension=2,
                 vdecay=0.5, vth=0.5, snn_timestep=20):
        """
        Args:
            input_2_hidden_weight (ndarray): weights for connection between input and hidden layer
            hidden_2_output_weight (ndarray): weights for connection between hidden and output layer
            input_dimension (int): input dimension
            hidden_dimension (int): hidden_dimension
            output_dimension (int): output_dimension
            vdecay (float): voltage decay of LIF neuron
            vth (float): voltage threshold of LIF neuron
            snn_timestep (int): number of timesteps for inference
        """
        self.snn_timestep = snn_timestep
        self.output_layer = LIFNeurons(output_dimension, vdecay, vth)
        self.input_2_output_connection = Connections(input_2_output_weight, input_dimension, output_dimension)
    
    def __call__(self, spike_encoding):
        """
        Args:
            spike_encoding (ndarray): spike encoding of input
        Return:
            spike outputs of the network
        """
        spike_output = np.zeros(self.output_layer.dimension)
        for tt in range(self.snn_timestep):
            input_2_output_psp = self.input_2_output_connection(spike_encoding[:, tt])
            output_spikes = self.output_layer(input_2_output_psp)
            spike_output += output_spikes
        return spike_output/self.snn_timestep      

## 2c. 
Next, you need to write a function for network training using hebbian learning rule. The function is defined below. You need to fill in the components so that the network weights are updated in the right manner. 

In [54]:
def hebbian(network, train_data, lr=1e-5, epochs=10):
    """ 
    Function to train a network using Hebbian learning rule
        Args:
            network (SNN): SNN network object
            train_data (list): training data 
            lr (float): learning rate
            epochs (int): number of epochs to train with. Each epoch is defined as one pass over all training samples. 
        
        Write the operations required to compute the weight increment according to the hebbian learning rule. Then increment the network weights. 
    """
    
    #iterate over the epochs
    for ee in range(epochs):
        #iterate over all samples in train_data
        w = np.zeros((1, 2))
        for data in train_data:
            #compute the firing rate for the input
            ri_1 = np.mean(data[0][0])
            ri_2 = np.mean(data[0][1])

            #compute the firing rate for the output
            ro = np.mean(data[1])

            #compute the correlation using the firing rates calculated above
            c1 = ri_1 * ro
            c2 = ri_2 * ro

            #compute the weight increment
            w[0][0] = c1 * lr
            w[0][1] = c2 * lr

            #increment the weight
            network.input_2_output_connection.weights += w


## 2d. 
In this exercise, you will use your implementations above to train an SNN to learn AND gate. 

In [56]:
#Define a variable for input dimension
input_dimension = 2

#Define a variable for output dimension
output_dimension = 1

#Define a variable for voltage decay
vdecay = 0.5

#Define a variable for voltage threshold
vth = 0.5

#Define a variable for snn timesteps
snn_timestep = 20

#Initialize randomly the weights from input to output. Numpy random rand function might be useful here. 
input_2_output_weight = np.random.rand(1, 2) / 10

#print the initial weights
print(input_2_output_weight)

#Initialize an snn using the arguments defined above
snn = SNN(input_2_output_weight, input_dimension, output_dimension, vdecay, vth, snn_timestep)

#Get the training data for AND gate using the function defined in 2a. 
train_data = genANDTrainData(snn_timestep)

#Train the network using the function defined in 2c. with the appropriate arguments
hebbian(snn, train_data, lr = 2e-2, epochs = 20)

#Test the trained network and print the network output for all 4 cases. 
output = []
for i, data in enumerate(train_data):
    out = snn(data[0])
    print("Case ", i+1, " : ", out)

    if out > vth:
        output.append(1)
    else:
        output.append(0)

print(output)

#Print Final Network Weights
print(snn.input_2_output_connection.weights)

[[0.07703541 0.0112078 ]]
Case  1  :  [0.4]
Case  2  :  [0.25]
Case  3  :  [0.3]
Case  4  :  [0.6]
[0, 0, 0, 1]
[[0.31703541 0.2512078 ]]


# Question 3: Limitations of Hebbian Learning rule

## 3a. 
Can you learn the AND gate using 2 neurons in the output layer instead of one? If yes, describe what changes you might need to make to your algorithm in 2b. If not, explain why not, and what consequences it might entail for the use of hebbian learning for complex real-world tasks. 

## Answer 3a. 
It really makes no sense to use two neurons for something as simple as an AND gate. The AND gate is a simple function that can be learned by a single neuron. However, if you were to use two neurons, you would need to change the decoding function to take the max of the two neurons.

## 3b. 
Train the network using hebbian learning for AND gate with the same arguments as defined in 2d. but now multiply the number of epochs by 20. Can your network still learn AND gate correctly? Inspect the initial and final network weights, and compare them against the network weights in 2d. Based on this, explain your observations for the network behavior. 

In [57]:
#Define a variable for input dimension
input_dimension = 2

#Define a variable for output dimension
output_dimension = 1

#Define a variable for voltage decay
vdecay = 0.5

#Define a variable for voltage threshold
vth = 0.5

#Define a variable for snn timesteps
snn_timestep = 20

#Initialize randomly the weights from input to output. Numpy random rand function might be useful here. 
input_2_output_weight = np.random.rand(1, 2) / 10

#print the initial weights
print(input_2_output_weight)

#Initialize an snn using the arguments defined above
snn = SNN(input_2_output_weight, input_dimension, output_dimension, vdecay, vth, snn_timestep)

#Get the training data for AND gate using the function defined in 2a. 
train_data = genANDTrainData(snn_timestep)

#Train the network using the function defined in 2c. with the appropriate arguments
hebbian(snn, train_data, lr = 2e-2, epochs = 400)

#Test the trained network and print the network output for all 4 cases. 
output = []
for i, data in enumerate(train_data):
    out = snn(data[0])
    print("Case ", i+1, " : ", out)

    if out > vth:
        output.append(1)
    else:
        output.append(0)

print(output)

#Print Final Network Weights
print(snn.input_2_output_connection.weights)

[[0.05525041 0.07584377]]
Case  1  :  [0.25]
Case  2  :  [0.6]
Case  3  :  [0.6]
Case  4  :  [0.5]
[0, 1, 1, 0]
[[4.05525041 4.07584377]]


## Answer 3b. 
After 400 epochs it looks like the network fails to learn the AND gate. Moreover it looks like the network has oversaturated similar to what we observed in the first assignment. We can likely fix this by using a learning rate that decays over time again similar to what we used in the first assignment.

## 3c. 
Based on your observations and response in 3b., can you explain another limitation of hebbian learning rule w.r.t. weight growth? Can you also suggest a possible remedy for it?

## Answer 3c. 
There is no fix to the weight growth problem in Hebbian learning. The weights will always grow and saturate. This is a fundamental limitation of Hebbian learning. We can fix this by using Oja's rule. Oja's rule is a variant of Hebbian learning that normalizes the weights to prevent them from growing too large. This change allows for a more stable weight update.

## 3d. 
To resolve the issues with hebbian learning, one possibility is Oja's rule. In this exercise, you will implement and train an SNN using Oja's learning rule. 

In [58]:
def oja(network, train_data, lr=1e-5, epochs=10):
    """ 
    Function to train a network using Hebbian learning rule
        Args:
            network (SNN): SNN network object
            train_data (list): training data 
            lr (float): learning rate
            epochs (int): number of epochs to train with. Each epoch is defined as one pass over all training samples. 
        
        Write the operations required to compute the weight increment according to the hebbian learning rule. Then increment the network weights. 
    """
    
    #iterate over the epochs
    for ee in range(epochs):
        #iterate over all samples in train_data
        w = np.zeros((1, 2))
        for data in train_data:
            #compute the firing rate for the input
            ri_1 = np.mean(data[0][0])
            ri_2 = np.mean(data[0][1])

            #compute the firing rate for the output
            ro = np.mean(data[1])

            #compute the correlation using the firing rates calculated above
            c1 = ri_1 * ro
            c2 = ri_2 * ro

            ot_1 = network.input_2_output_connection.weights[0][0] * ro
            ot_2 = network.input_2_output_connection.weights[0][1] * ro

            # #compute the weight increment
            w[0][0] = (c1 - ot_1) * lr * ro
            w[0][1] = (c2 - ot_2) * lr * ro

            #increment the weight
            network.input_2_output_connection.weights += w


Now, test your implementation below. 

In [114]:
#Define a variable for input dimension
input_dimension = 2

#Define a variable for output dimension
output_dimension = 1

#Define a variable for voltage decay
vdecay = 0.5

#Define a variable for voltage threshold
vth = 0.5

#Define a variable for snn timesteps
snn_timestep = 20

#Initialize randomly the weights from input to output. Numpy random rand function might be useful here. 
input_2_output_weight = np.random.rand(1, 2) / 10

#print the initial weights
print(input_2_output_weight)

#Initialize an snn using the arguments defined above
snn = SNN(input_2_output_weight, input_dimension, output_dimension, vdecay, vth, snn_timestep)

#Get the training data for AND gate using the function defined in 2a. 
train_data = genANDTrainData(snn_timestep)

#Train the network using the function defined in 3d. with the appropriate arguments
oja(snn, train_data, lr = 2e-3, epochs = 400)

#Test the trained network and print the network output for all 4 cases. 
output = []
for i, data in enumerate(train_data):
    out = snn(data[0])
    print("Case ", i+1, " : ", out)

    if out > vth:
        output.append(1)
    else:
        output.append(0)

print(output)

#Print Final Network Weights
print(snn.input_2_output_connection.weights)

[[0.07595651 0.03276177]]
Case  1  :  [0.4]
Case  2  :  [0.5]
Case  3  :  [0.5]
Case  4  :  [0.75]
[0, 0, 0, 1]
[[0.44737526 0.42798215]]


# Question 4: Spike-time dependent plasticity (STDP)

## 4a. 
What is the limitation with hebbian learning that STDP aims to resolve?

## Answer 4a. 
An issue in hebbian learning is that the association between two neurons is not strengthened if they happen around the same time. STDP aims to fix this by strengthening the association between two neurons if they fire close to each other in time.

## 4b. 
Describe the algorithm to train a network using STDP learning rule. You do not need to describe encoding here. Your algorithm should be such that its naturally translatable to a program. 

## Answer 4b. 
STDP uses pre and post synaptic spikes to update the weights. If the pre-synaptic spike happens before the post-synaptic spike, the weight is increased. If the post-synaptic spike happens before the pre-synaptic spike, the weight is decreased. This is a simple learning rule that is biologically plausible.

In the network the connections have time delays. The time delays are used to calculate the time difference between the pre and post synaptic spikes. If the time difference is positive, the weight is increased. If the time difference is negative, the weight is decreased.

## 4c. 
In this exercise, you will implement the STDP learning algorithm to train a network. STDP has many different flavors. For this exercise, we will use the learning rule defined in: https://dl.acm.org/doi/pdf/10.1609/aaai.v33i01.330110021. Pay special attention to Equations 2 and 3. 

Below is the class definition for STDP learning algorithm. Your task is to fill in the components so that the weights are updated in the right manner. 

In [4]:
class STDP():
    """Train a network using STDP learning rule"""
    def __init__(self, network, A_plus, A_minus, tau_plus, tau_minus, lr, snn_timesteps=20, epochs=30, w_min=0, w_max=1):
        """
        Args:
            network (SNN): network which needs to be trained
            A_plus (float): STDP hyperparameter
            A_minus (float): STDP hyperparameter
            tau_plus (float): STDP hyperparameter
            tau_minus (float): STDP hyperparameter
            lr (float): learning rate
            snn_timesteps (int): SNN simulation timesteps
            epochs (int): number of epochs to train with. Each epoch is defined as one pass over all training samples.  
            w_min (float): lower bound for the weights
            w_max (float): upper bound for the weights
        """
        self.network = network
        self.A_plus = A_plus
        self.A_minus = A_minus
        self.tau_plus = tau_plus
        self.tau_minus = tau_minus
        self.snn_timesteps = snn_timesteps
        self.lr = lr
        self.time = np.arange(0, self.snn_timesteps, 1)
        self.sliding_window = np.arange(-4, 4, 1) #defines a sliding window for STDP operation. 
        self.epochs = epochs
        self.w_min = w_min
        self.w_max = w_max
    
    def update_weights(self, t, i):
        """
        Function to update the network weights using STDP learning rule
        
        Args:
            t (int): time difference between postsynaptic spike and a presynaptic spike in a sliding window
            i(int): index of the presynaptic neuron
        
        Fill the details of STDP implementation
        """
        #compute delta_w for positive time difference
        if t>0:
            delta_w = self.A_plus * math.exp(-t/self.tau_plus)
        #compute delta_w for negative time difference
        else:
            delta_w = -self.A_minus * math.exp(-t/self.tau_minus)
            
        #update the network weights if weight increment is negative
        if delta_w < 0:
            change = self.lr * delta_w * (self.network.input_2_output_connection.weights - self.w_min)
            self.network.input_2_output_connection.weights += change
        #update the network weights if weight increment is positive
        elif delta_w > 0:
            change = self.lr * delta_w * (self.w_max - self.network.input_2_output_connection.weights)
            self.network.input_2_output_connection.weights += change
            
    def train_step(self, train_data_sample):
        """
        Function to train the network for one training sample using the update function defined above. 
        
        Args:
            train_data_sample (list): a sample from the training data
            
        This function is complete. You do not need to do anything here. 
        """
        input = train_data_sample[0]
        output = train_data_sample[1]
        for t in self.time:
            if output[t] == 1:
                for i in range(2):
                    for t1 in self.sliding_window:
                        if (0<= t + t1 < self.snn_timesteps) and (t1!=0) and (input[i][t+t1] == 1):
                            self.update_weights(t1, i)
    
    def train(self, training_data):
        """
        Function to train the network
        
        Args:
            training_data (list): training data
        
        This function is complete. You do not need to do anything here. 
        """
        for ee in range(self.epochs):
            for train_data_sample in training_data:
                self.train_step(train_data_sample)

Let's test the implementation

In [10]:
#Define a variable for input dimension
input_dimension = 2

#Define a variable for output dimension
output_dimension = 1

#Define a variable for voltage decay
vdecay = 0.5

#Define a variable for voltage threshold
vth = 0.5

#Define a variable for snn timesteps
snn_timestep = 20

#Initialize randomly the weights from input to output. Numpy random rand function might be useful here. 
input_2_output_weight = np.random.rand(1, 2)
print(input_2_output_weight)

#Initialize an snn using the arguments defined above
snn = SNN(input_2_output_weight, input_dimension, output_dimension, vdecay, vth, snn_timestep)

#Get the training data for AND gate using the function defined in 2a. 
train_data = genANDTrainData(snn_timestep)

#Create an object of STDP class with appropriate arguments
stdp = STDP(snn, A_plus=0.6, A_minus=0.3, tau_plus=8, tau_minus=5, lr=0.25, snn_timesteps=5, epochs=30, w_min=0, w_max=1)

#Train the network using STDP
stdp.train(train_data)

#Test the trained network and print the network output for all 4 cases. 
output = []
for i, data in enumerate(train_data):
    out = snn(data[0])
    print("Case ", i+1, " : ", out)

    if out > vth:
        output.append(1)
    else:
        output.append(0)

print(output)

[[0.81561567 0.62389058]]
Case  1  :  [0.3]
Case  2  :  [0.45]
Case  3  :  [0.45]
Case  4  :  [0.65]
[0, 0, 0, 1]


# Question 5: OR Gate
Can you train the network with the same architecture in Q2-4 for learning the OR gate. You will need to create another function called genORTrainData. Then create an SNN and train it using STDP. 

In [11]:
def genORTrainData(snn_timestep):
    """ 
    Function to generate the training data for AND 
        Args:
            snn_timestep (int): timesteps for SNN simulation
        Return:
            train_data (list): list of tuples where each tuple comprises of numpy arrays of rate-coded inputs and output
        
        Write the expressions for encoding 0 and 1. Then append all 4 cases of AND gate to the list train_data
    """
    
    #Initialize an empty list for train data
    train_data = []
    
    #encode 0. Numpy random choice function might be useful here. 
    z = np.random.choice([0,1], size=snn_timestep, p=[0.7, 0.3])
    
    #encode 1. Numpy random choice function might be useful here. 
    o = np.random.choice([0,1], size=snn_timestep, p=[0.3, 0.7])
    
    #Append all 4 cases of AND gate to train_data. Numpy stack operation might be useful here. 
    train_data.append((np.stack((z, z), axis=0), z))
    train_data.append((np.stack((z, o), axis=0), o))
    train_data.append((np.stack((o, z), axis=0), o))
    train_data.append((np.stack((o, o), axis=0), o))

    return train_data

In [13]:
#Define a variable for input dimension
input_dimension = 2

#Define a variable for output dimension
output_dimension = 1

#Define a variable for voltage decay
vdecay = 0.5

#Define a variable for voltage threshold
vth = 0.5

#Define a variable for snn timesteps
snn_timestep = 20

#Initialize randomly the weights from input to output. Numpy random rand function might be useful here. 
input_2_output_weight = np.random.rand(1, 2)
print(input_2_output_weight)

#Initialize an snn using the arguments defined above
snn = SNN(input_2_output_weight, input_dimension, output_dimension, vdecay, vth, snn_timestep)

#Get the training data for AND gate using the function defined in 2a. 
train_data = genORTrainData(snn_timestep)

#Create an object of STDP class with appropriate arguments
stdp = STDP(snn, A_plus=0.6, A_minus=0.3, tau_plus=8, tau_minus=5, lr=0.25, snn_timesteps=5, epochs=30, w_min=0, w_max=1)

#Train the network using STDP
stdp.train(train_data)

#Test the trained network and print the network output for all 4 cases. 
output = []
for i, data in enumerate(train_data):
    out = snn(data[0])
    print("Case ", i+1, " : ", out)

    if out > vth:
        output.append(1)
    else:
        output.append(0)

print(input_2_output_weight)

print(output)

[[0.63853067 0.24668528]]
Case  1  :  [0.15]
Case  2  :  [0.2]
Case  3  :  [0.2]
Case  4  :  [0.7]
[[0.28751783 0.28751783]]
[0, 0, 0, 1]
